### Here we build a basic RAG chatbot using Ollama Nomic Embeddings, Llama2 70b from Groq LPU, LangChain, Gradio
### For the data sources we use articles on The Cosmic Microwave Background Radiation (CMB)::
* https://arxiv.org/abs/1210.6008v1 - CMB Review, Challinor, 2012
* https://arxiv.org/abs/1606.03606 - CMB Foreground review, Challinson, 2016
* https://arxiv.org/abs/0803.0834 - CMB Review, Samtleben at al., 2008

You should have a .env file in the environment in which you are running this notebook/script, which contains the line
GROQ_API_KEY='Your API Key here'

Ollama should also be running on your local machine

#### References:: 
* https://www.linkedin.com/pulse/build-lightning-fast-rag-chatbot-powered-groqs-lpu-ollama-multani-ssloc/
* https://colab.research.google.com/drive/1Obrby8RniFfjUvf3DhbNHC6-CmBdiXbY?usp=sharing


In [7]:
# Get the Nomic embeddings

!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 1

In [3]:
import gradio as gr
import os
import time
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import textwrap

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai_api_key = os.environ['OPENAI_API_KEY'] # Not needed for this example
# hf_api_key = os.environ['HF_API_KEY'] # Not needed for this example

groq_api_key = os.environ['GROQ_API_KEY']

### Define the RAG Components

In [10]:
# Load PDF documents from the 'data' directory
loader = PyPDFDirectoryLoader("../data")
the_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
chunks = text_splitter.split_documents(the_text)

In [11]:
# Setup the vector store and retriever
vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name="ollama_embeds",
    embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

In [12]:
# Get the LLM (Llama2-70b) from Groq

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name='llama2-70b-4096'
)

In [13]:
def process_question(question):

    # Define the RAG template and chain
    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke(question)

In [14]:
# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(label='User Question', lines=2, placeholder="Type your question here... "),
                     outputs=gr.Textbox(label='LLM Response'),
                     title="A Chat about the Oldest Light in the Universe",
                     description="Ask any question about The Cosmic Microwave Background Radiation. It's niche but it's magical! ",
                    )

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
